In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist, cifar10
import numpy as np

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
X_train = X_train.reshape(-1, 28*28).astype(np.float32) / 255
X_test = X_test.reshape(-1, 28*28).astype(np.float32) / 255

In [5]:
# Sequential API. Very convenient but not very flexible

model = keras.Sequential(
  [
    keras.Input(shape=(28*28)),
    layers.Dense(512, activation="relu"),
    layers.Dense(256, activation="relu"),
    layers.Dense(10)
  ]
)

model.compile(
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=keras.optimizers.Adam(lr=0.001),
  metrics=["accuracy"]
)

model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=True)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3119 - accuracy: 0.9052
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0811 - accuracy: 0.9750
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0494 - accuracy: 0.9839
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0406 - accuracy: 0.9864
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0301 - accuracy: 0.9908


In [8]:
# Another Method to build Keras Model using Sequential API
model2 = keras.Sequential()
model2.add(keras.Input(shape=(28*28)))
model2.add(layers.Dense(512, activation="relu"))
model2.add(layers.Dense(256, activation="relu"))
model2.add(layers.Dense(10))

model2.compile(
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=keras.optimizers.Adam(lr=0.001),
  metrics=["accuracy"]
)

model2.fit(X_train, y_train, batch_size=32, epochs=5, verbose=True)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3179 - accuracy: 0.9034
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0853 - accuracy: 0.9724
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0513 - accuracy: 0.9835
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0368 - accuracy: 0.9881
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0286 - accuracy: 0.9908


In [13]:
# Extract Output of a particular layer in a model

model2_temp = keras.Model(inputs=model2.inputs, outputs=model2.layers[-2].output)   # Output of second last layer
print(model2_temp.predict(X_train).shape,"\n")


model2_temp1 = keras.Model(inputs=model2.inputs, outputs = [layer.output for layer in model2.layers])
features = model2_temp1.predict(X_train)
for feature in features:
  print(feature.shape)

(60000, 256) 

(60000, 512)
(60000, 256)
(60000, 10)


In [9]:
# Functional API  (More Flexible)

inputs = keras.Input(shape=(28*28))
X = layers.Dense(512,activation="relu")(inputs)
X = layers.Dense(256, activation="relu")(X)
outputs = layers.Dense(10, activation="softmax")(X)

model3 = keras.Model(inputs=inputs, outputs=outputs)

model3.compile(
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
  optimizer=keras.optimizers.Adam(lr=0.001),
  metrics=["accuracy"]
)

model3.fit(X_train, y_train, batch_size=32, epochs=5, verbose=True)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3219 - accuracy: 0.8986
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0839 - accuracy: 0.9729
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0529 - accuracy: 0.9832
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0377 - accuracy: 0.9878
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0285 - accuracy: 0.9906


In [3]:
# Convolution

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [4]:
def my_model():

    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, 3, padding="same", kernel_regularizer=regularizers.l2(0.01))(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, padding="same", kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(
        128, 3, padding="same", kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model_cnn = my_model()
model_cnn.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"],
)

In [10]:
model_cnn.fit(x_train, y_train, batch_size=64, epochs=5, verbose=2)
model_cnn.evaluate(x_test, y_test, batch_size=64, verbose=True)

Epoch 1/5
782/782 - 7s - loss: 1.2549 - accuracy: 0.6332
Epoch 2/5
782/782 - 6s - loss: 1.2334 - accuracy: 0.6455
Epoch 3/5
782/782 - 6s - loss: 1.2182 - accuracy: 0.6500
Epoch 4/5
782/782 - 6s - loss: 1.2149 - accuracy: 0.6533
Epoch 5/5
782/782 - 6s - loss: 1.1982 - accuracy: 0.6586
157/157 [==============================] - 1s 5ms/step - loss: 1.0882 - accuracy: 0.7127


[1.088179349899292, 0.7127000093460083]